In [ ]:
library(msigdbr)
library(fgsea)
m_df<- msigdbr(species = "Homo sapiens", category = "H")
go_df = msigdbr(species = "Homo sapiens", category = "C7", subcategory = "IMMUNESIGDB")
fgsea_sets<- m_df %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:


chr_names = names(list_DE_res_base)[1:25]
list_res_gsea_base = vector(mode = "list", length = 5L)
names(list_res_gsea_base) = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD")
list_res_gsea_base$`VAF>=0.1-LUSC` = vector(mode = "list", length = 25L)
list_res_gsea_base$`0.1>VAF>=0.02-LUSC` = vector(mode = "list", length = 25L)
list_res_gsea_base$`Negative-LUSC` = vector(mode = "list", length = 25L)
list_res_gsea_base$`0.1>VAF>=0.02-LUAD` = vector(mode = "list", length = 25L)
list_res_gsea_base$`Negative-LUAD` = vector(mode = "list", length = 25L)

names(list_res_gsea_base$`VAF>=0.1-LUSC`) = chr_names 
names(list_res_gsea_base$`0.1>VAF>=0.02-LUSC`) = chr_names 
names(list_res_gsea_base$`Negative-LUSC`) = chr_names 
names(list_res_gsea_base$`0.1>VAF>=0.02-LUAD`) = chr_names 
names(list_res_gsea_base$`Negative-LUAD`) = chr_names 

list_rank_base = vector(mode = "list", length = 5L)
names(list_rank_base) = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD")
list_rank_base$`VAF>=0.1-LUSC` = vector(mode = "list", length = 25L)
list_rank_base$`0.1>VAF>=0.02-LUSC` = vector(mode = "list", length = 25L)
list_rank_base$`Negative-LUSC` = vector(mode = "list", length = 25L)
list_rank_base$`0.1>VAF>=0.02-LUAD` = vector(mode = "list", length = 25L)
list_rank_base$`Negative-LUAD` = vector(mode = "list", length = 25L)
names(list_rank_base$`VAF>=0.1-LUSC`)= chr_names 
names(list_rank_base$`0.1>VAF>=0.02-LUSC`)=chr_names 
names(list_rank_base$`Negative-LUSC`)= chr_names 
names(list_rank_base$`0.1>VAF>=0.02-LUAD`)= chr_names 
names(list_rank_base$`Negative-LUAD`)= chr_names 

for (each_name in chr_names) {
  print(paste0(each_name, " cluster is processing"))
  list_gsea = list_DE_res_base[[each_name]] %>% 
    separate(group, sep = "-", into = c("VAF_bin", "cluster", "time", "pathology")) %>%
    mutate(group = paste0(VAF_bin,"-", pathology)) %>%
    filter(pathology %in% c("LUAD", "LUSC"), avgExpr > 0.3) %>%
    mutate(group = factor(group, levels = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD"))) %>%
    group_by(group) %>% 
    arrange(-auc, -abs(logFC)) %>%
    select(group, feature, auc) 
  
  c_names_list_gsea = list_gsea %>% group_keys() %>% pull(group)
  list_gsea = list_gsea %>% group_split(.keep = F)
  names(list_gsea) = c_names_list_gsea
  
  
  df_gsea_stat = list_DE_res_base[[each_name]] %>% 
    separate(group, sep = "-", into = c("VAF_bin", "cluster", "time", "pathology")) %>%
    mutate(group = paste0(VAF_bin,"-", pathology)) %>%
    filter(pathology %in% c("LUAD", "LUSC")) %>%
    group_by(group) %>% 
    arrange(-auc, -abs(logFC)) %>%
    select(group, feature, logFC, auc, padj)
  
  p1 = df_gsea_stat %>% 
    mutate(group = factor(group, levels = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD"))) %>%
    ggplot(aes(x = auc, fill = group)) + 
    geom_histogram(binwidth = 0.005) + 
    scale_fill_igv() +
    scale_y_log10()+
    theme_pubr(legend = "none") +
    xlim(0.3,0.8)
  
  p2 = df_gsea_stat %>% 
    mutate(group = factor(group, levels = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD"))) %>%
    filter(padj < 1e-3) %>%
    ggplot(aes(x = logFC, y = -log10(padj))) +
    scale_color_igv() +
    geom_point(aes(color = group)) +
    theme_pubr(legend = "bottom")
  
  
  for (idx in 1:5){
    #rank =  list_gsea[[idx]] %>% filter(!auc == 0.5) %>% mutate(auc = auc - 0.5) %>% deframe()
    rank =  list_gsea[[idx]] %>% filter(auc > 0.6 | auc < 0.4) %>% mutate(auc = auc - 0.5) %>% deframe()
    fgsea_res = fgsea(fgsea_sets,rank, eps = 0)
    list_rank_base[[idx]][[each_name]] = rank
    list_res_gsea_base[[idx]][[each_name]] = fgsea_res
  }

}


In [ ]:

chr_names = names(list_DE_res_1st)[1:25]
list_res_gsea_1st = vector(mode = "list", length = 5L)
names(list_res_gsea_1st) = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD")
list_res_gsea_1st$`VAF>=0.1-LUSC` = vector(mode = "list", length = 25L)
list_res_gsea_1st$`0.1>VAF>=0.02-LUSC` = vector(mode = "list", length = 25L)
list_res_gsea_1st$`Negative-LUSC` = vector(mode = "list", length = 25L)
list_res_gsea_1st$`0.1>VAF>=0.02-LUAD` = vector(mode = "list", length = 25L)
list_res_gsea_1st$`Negative-LUAD` = vector(mode = "list", length = 25L)

names(list_res_gsea_1st$`VAF>=0.1-LUSC`) = chr_names 
names(list_res_gsea_1st$`0.1>VAF>=0.02-LUSC`) = chr_names 
names(list_res_gsea_1st$`Negative-LUSC`) = chr_names 
names(list_res_gsea_1st$`0.1>VAF>=0.02-LUAD`) = chr_names 
names(list_res_gsea_1st$`Negative-LUAD`) = chr_names 

list_rank_1st = vector(mode = "list", length = 5L)
names(list_rank_1st) = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD")
list_rank_1st$`VAF>=0.1-LUSC` = vector(mode = "list", length = 25L)
list_rank_1st$`0.1>VAF>=0.02-LUSC` = vector(mode = "list", length = 25L)
list_rank_1st$`Negative-LUSC` = vector(mode = "list", length = 25L)
list_rank_1st$`0.1>VAF>=0.02-LUAD` = vector(mode = "list", length = 25L)
list_rank_1st$`Negative-LUAD` = vector(mode = "list", length = 25L)
names(list_rank_1st$`VAF>=0.1-LUSC`)= chr_names 
names(list_rank_1st$`0.1>VAF>=0.02-LUSC`)=chr_names 
names(list_rank_1st$`Negative-LUSC`)= chr_names 
names(list_rank_1st$`0.1>VAF>=0.02-LUAD`)= chr_names 
names(list_rank_1st$`Negative-LUAD`)= chr_names 

for (each_name in chr_names) {
  print(paste0(each_name, " cluster is processing"))
  list_gsea = list_DE_res_1st[[each_name]] %>% 
    separate(group, sep = "-", into = c("VAF_bin", "cluster", "time", "pathology")) %>%
    mutate(group = paste0(VAF_bin,"-", pathology)) %>%
    filter(pathology %in% c("LUAD", "LUSC"), avgExpr > 0.3) %>%
    mutate(group = factor(group, levels = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD"))) %>%
    group_by(group) %>% 
    arrange(-auc, -abs(logFC)) %>%
    select(group, feature, auc) 
  
  c_names_list_gsea = list_gsea %>% group_keys() %>% pull(group)
  list_gsea = list_gsea %>% group_split(.keep = F)
  names(list_gsea) = c_names_list_gsea
  
  
  df_gsea_stat = list_DE_res_1st[[each_name]] %>% 
    separate(group, sep = "-", into = c("VAF_bin", "cluster", "time", "pathology")) %>%
    mutate(group = paste0(VAF_bin,"-", pathology)) %>%
    filter(pathology %in% c("LUAD", "LUSC")) %>%
    group_by(group) %>% 
    arrange(-auc, -abs(logFC)) %>%
    select(group, feature, logFC, auc, padj)
  
  p1 = df_gsea_stat %>% 
    mutate(group = factor(group, levels = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD"))) %>%
    ggplot(aes(x = auc, fill = group)) + 
    geom_histogram(binwidth = 0.005) + 
    scale_fill_igv() +
    scale_y_log10()+
    theme_pubr(legend = "none") +
    xlim(0.3,0.8)
  
  p2 = df_gsea_stat %>% 
    mutate(group = factor(group, levels = c("VAF>=0.1-LUSC", "0.1>VAF>=0.02-LUSC", "Negative-LUSC", "0.1>VAF>=0.02-LUAD", "Negative-LUAD"))) %>%
    filter(padj < 1e-3) %>%
    ggplot(aes(x = logFC, y = -log10(padj))) +
    scale_color_igv() +
    geom_point(aes(color = group)) +
    theme_pubr(legend = "bottom")
  
  
  for (idx in 1:5){
    rank =  list_gsea[[idx]] %>% filter(!auc == 0.5) %>% mutate(auc = auc - 0.5) %>% deframe()
    fgsea_res = fgsea(fgsea_sets,rank, eps = 0)
    list_rank_1st[[idx]][[each_name]] = rank
    list_res_gsea_1st[[idx]][[each_name]] = fgsea_res
  }
  
  
  p3 = list_res_gsea_1st$`VAF>=0.1-LUSC`[[each_name]] %>% as_tibble() %>%  arrange(abs(NES)) %>%
    mutate(pathway = factor(pathway, levels = unique(pathway)),
           color = ifelse(NES > 0 & padj < 0.05, "significant_high",
                          ifelse(NES < 0 & padj < 0.05, "significant_low", "ns"))) %>%
    ggplot(aes(x = NES, y = pathway, fill = color)) + geom_bar(stat = "identity")  + scale_fill_manual(values = c("grey80","#FF4422", "#3366FF"))
  each_fname = paste0("/path/to/202111-CHIP/figure_prep/DE_stats/230810_1st_DE_", each_name, ".png")
  ggarrange(p1+ ggtitle(paste0(each_name, " DE + GSEA distribution, time = 1st")), p2, p3, ncol = 1, heights = c(1,1,2)) %>%
    ggsave(plot = ., filename = each_fname, height = 15, width = 10)
  
  
  print(paste0(each_name, " is saved in ", each_fname))
}

for (each_idx in 1:length(list_res_gsea_base$`Negative-LUSC`)) {
  list_res_gsea_base$`Negative-LUSC`[[each_idx]]$cluster = names(list_res_gsea_base$`Negative-LUSC`)[each_idx] 
}
for (each_idx in 1:length(list_res_gsea_1st$`Negative-LUSC`)) {
  list_res_gsea_1st$`Negative-LUSC`[[each_idx]]$cluster = names(list_res_gsea_1st$`Negative-LUSC`)[each_idx] 
}
for (each_idx in 1:length(list_res_gsea_1st$`VAF>=0.1-LUSC`)) {
  list_res_gsea_1st$`VAF>=0.1-LUSC`[[each_idx]]$cluster = names(list_res_gsea_1st$`VAF>=0.1-LUSC`)[each_idx] 
}